# Efficiency: caching (continued); loop optimization
_COSC 208, Introduction to Computer Systems, 2023-04-10_

## Announcements
* DEI assignment 3 due Thursday @ 11pm
* Project 4 due Thursday, April 20

## Outline
* Warm-up
* Other instances of caching
* Cache replacement
* Optimizing loops for locality

## Warm-up

Q1: _For each of the following scenarios, indicate whether it is an example of temporal locality, spatial locality, or neither._

* Gates for flights on the same airline are located in the same airport terminal/concourse – spatial locality
* A grocery list is arranged in alphabetical order – neither
* Clothes in a closet are grouped into outfits, with a shirt and a pair of pants stored next to each other – spatial locality
* Boxes of cereal, bowls, and spoons are stored in adjacent kitchen cabinets/drawers – spatial locality
* You repeatedly check your phone for new messages – temporal locality
* A variable used in a for loop – temporal locality
* Variables used in different functions – neither
* A function's parameters, which are each used once within the function – spatial locality

* Gates for flights on the same airline are located in the same airport terminal/concourse
* A grocery list is arranged in alphabetical order
* Clothes in a closet are grouped into outfits, with a shirt and a pair of pants stored next to each other
* Boxes of cereal, bowls, and spoons are stored in adjacent kitchen cabinets/drawers
* You repeatedly check your phone for new messages
* A variable used in a for loop
* Variables used in different functions
* A function's parameters, which are each used once within the function

🛑 **STOP here** after completing the above question; if you have extra time take a few deep breaths to reduce stress.

## Other instances of caching

* _Where are caches used in computer systems besides the CPU?_
    * Operating systems
    * Web browsers
    * Web servers
    * Domain Name System (DNS)
    * Databases
    * Hard Disk Drives (HDDs)
    * Solid State Drives (SSDs)

* Web browser caches
    * _Why do web browsers have caches?_
        * Accessing remote network storage is >50x slower than accessing a solid state drive (SSD)
        * Spatial locality — many aspects of a web page are also used with other pages on the same site: e.g., images, Cascading Style Sheets (CSS), JavaScript (JS)
        * Temporal locality — users often visit the same web page repeatedly: e.g., Google
        * Internet Service Provider (ISP) may limit amount of data downloaded/uploaded per month
    * Store static content (e.g., images, CSS, JS)
    * Web browser caches are read-only
* Content distribution networks (CDNs)
    * Collection of geographically distributed servers that delivery content (e.g., streaming videos) to users
    * User's computers contact a server that is "nearby"
        * Ideally measured in terms of latency, which is a function of geographic distance, network routes, and network load
        * Analogy: time it takes to drive somewhere is a function of geographic distance, the route you take, and the amount of traffic on the road
    * CDN servers fetch and cache content from origin servers
    * Popular content (e.g., image from the front page of the NY Times) is more likely to already be cached
* Domain Name System (DNS)
    * Web browser, operating system, and/or DNS server cache mappings from domain names (e.g., `portal.colgate.edu`) to Internet Protocol (IP) addresses (e.g., `149.43.134.29`)

## Cache replacement

* If a cache is full, then a cache entry must be removed so different data can be placed in the cache
* Cache replacement policy governs which data is removed
* _What should a good cache replacement policy do?_ — maximize the number of cache hits (or minimize the number of cache misses)
    * Evaluation metric: Hit ratio = number of hits / total number of memory accesses
* _How do we determine which cache entry to replace?_
* Optimal replacement policy – replace the entry that will be accessed furthest in the future
    * Impractical because we don’t know data access patterns a priori
* Least Recently Used (LRU)
    * LRU assumes a item that was accessed recently will be accessed again soon – temporal locality
    * Downside: lots of overhead to implement — need to store an ordered list of items and move an item up in the list whenever it’s accessed
    * Where does this go wrong? — when working-set size (i.e., number of repeatedly accessed entries) is (slightly) greater than size of the cache
* First-in First-out (FIFO)
    * Simple to implement
    * Doesn’t consider the importance of a cache entry
* Random
    * Even simpler to implement
    * Doesn’t consider the importance of a cache entry

* Assume a cache can hold 3 entries and the following 15 data accesses occur: 
```
3, 4, 4, 5, 3, 2, 3, 4, 1, 4, 4, 2, 5, 2, 4
```
* Q2: _What is the sequence of hits, insertions, and replacements that occur when an **optimal** cache replacement algorithm is used?_

```
+3, +4, H4, +5, H3, -5/+2, H3, H4, -3/+1, H4, H4, H2, -1/+5, H2, H4
Hit ratio = 9/15 = 60%
```

<p style="height:8em;"></p>

* Q3: _What is the sequence of hits, insertions, and replacements that occur when a **first in first out (FIFO)** cache replacement algorithm is used?_

```
+3, +4, H4, +5, H3, -3/+2, -4/+3, -5/+4, -2/+1, H4, H4, -3/+2, -4/+5, H2, -1/+4
Hit ratio = 5/15 = 33%
```

<p style="height:8em;"></p>

* Q4: _What is the sequence of hits, insertions, and replacements that occur when a **least recently used (LRU)** cache replacement algorithm is used?_

```
+3, +4, H4, +5, H3, -4/+2, H3, -5/+4, -2/+1, H4, H4, -3/+2, -1/+5, H2, H4
Hit ratio = 7/15 = 47%
```

<p style="height:8em;"></p>

🛑 **STOP here** after completing the above question; if you have extra time please **skip ahead** to the extra practice.

<div style="page-break-after:always;"></div>

## Optimizing loops for locality

* Techniques
    * Loop interchange — with nested loops, swap inner and outer loop
    * Loop fussion — combine two loops at the same level into a single loop
    * Loop fission — split a single loop into two loops at the same level

* Q5: _Modify the following function to improve spatial locality_

In [ ]:
void hundreds() {
    int *nums = malloc(sizeof(int) * 1000);
    for (int i = 0; i < 100; i++) {
        for (int j = 0; j < 1000; j+= 100) {
            nums[i+j] = i;
        }
    }
}

In [ ]:
// Perform loop interchange
void hundreds_optimized() {
    int *nums = malloc(sizeof(int) * 1000);
    for (int j = 0; j < 1000; j+= 100) {
        for (int i = 0; i < 100; i++) {
            nums[i+j] = i;
        }
    }
}

<p style="height:9em;"></p>

* Q6: _Modify the following function to improve temporal locality_

In [ ]:
int odds(int *nums, int length) {
    for (int i = 0; i < length; i++) {
        nums[i] = nums[i] % 2;
    }
    int count = 0;
    for (int j = 0; j < length; j++) {
        count += nums[j];
    }
    return count;
}

In [ ]:
// Perform loop fusion
int odds_optimized(int *nums, int length) {
    int count = =0;
    for (int i = 0; i < length; i++) {
        nums[i] = nums[i] % 2;
        count += nums[i];
    }
    return count;
}

<p style="height:10em;"></p>

* Q7: _Modify the following function to improve spatial locality_

In [ ]:
void multiplication(int grid[][], int rows, int cols) {
    for (int c = 0; c < cols; c++) {
        for (int r = 0; r < rows; r++) {
            grid[r][c] = c * r;
        }
    }
}

In [ ]:
// Perform loop interchange
void multiplication_optimized(int grid[][], int rows, int cols) {
    for (int r = 0; r < rows; r++) {
        for (int c = 0; c < cols; c++) {
            grid[r][c] = c * r;
        }
    }
}

<p style="height:8em;"></p>

* Q8: _Modify the following function to improve temporal locality_

In [ ]:
long stdev(int *nums, int length) {
    long sum = 0;
    for (int i = 0; i < length; i++) {
        sum += nums[i];
    }
    int mean = sum / length;
    sum = 0;
    for (int j = 0; j < length; j++) {
        int diff = nums[j] - mean;
        sum += diff * diff:
    }
    mean = sum / length;
    return sqrt(mean);
}

    It's not possible to optimize this code more—there are no loops to interchange; there are no loops to fission; the loops cannot be fused.

<p style="height:10em;"></p>

## Extra practice

Q9: _For each of the following instances of caching, indicate whether the caching is motivated by temporal or spatial locality._

* A CPU caches the first 32 instructions of a function when the function is called – spatial
* A CPU caches all of the instructions for a frequently called function – temporal
* A web browser caches the Moodle pages for your courses, which you view multiple times per week – temporal
* A content distribution network (CDN) caches a video that has gone viral – temporal
* A content distribution network (CDN) caches "recommended videos" related to a video – spatial

* A CPU caches the first 32 instructions of a function when the function is called
* A CPU caches all of the instructions for a frequently called function
* A web browser caches the Moodle pages for your courses, which you view multiple times per week
* A content distribution network (CDN) caches a video that has gone viral
* A content distribution network (CDN) caches "recommended videos" related to a video